In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot
import os
from copy import deepcopy

from time import time

from math import ceil
from scipy.stats import spearmanr, gamma, poisson

from anndata import AnnData, read_h5ad
import scanpy as sc
from scanpy import read
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from torch import tensor
from torch.cuda import is_available

from sciPENN.sciPENN_API import sciPENN_API

In [2]:
"""Read in Raw Data"""


adata_gene_train = sc.read("/home/asmauger/biostat625final/rna_train_trial.h5ad")
adata_gene_test = sc.read("/home/asmauger/biostat625final/rna_test_trial.h5ad")
adata_protein_train = sc.read("/home/asmauger/biostat625final/prot_train_trial.h5ad")
ref = sc.read("/home/asmauger/biostat625final/pbmc_gene.h5ad")

In [7]:
print(ref)
print(adata_gene_train)
sciPENN_API
print(adata_gene_train.obs['daydonor'])
import scanpy as sc
sc.pp.normalize_total(adata_gene_train)
sc.pp.log1p(adata_gene_train)
print(adata_gene_train.X[0:, 0:10])

AnnData object with n_obs × n_vars = 161764 × 20729
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase'
    var: 'name'
    obsm: 'X_apca', 'X_aumap', 'X_pca', 'X_spca', 'X_umap', 'X_wnn.umap'
AnnData object with n_obs × n_vars = 35496 × 1001
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'day', 'donor', 'cell_type', 'technology', 'daydonor', 'ident'
    uns: 'X_name'
    layers: 'logcounts'
2cbe27fc5780    day213176
f7cf4dfe9ead    day213176
b064f92b0732    day213176
ba4b38499423    day213176
e1689980982e    day213176
                  ...    
52c0f514e637    day432606
554915e7e85f    day432606
31f019585af0    day432606
0d39776f5ab5    day432606
6de4d955c537    day432606
Name: daydonor, Length: 35496, dtype: category
Categories (9, object): ['day213176', 'day231800', 'day232606', 'day313176', ..., 'day332606', 'day413176', 'day431800', 'day432606']
  (8, 0)	0.4

In [20]:
# note that donor, day are stored as integers and might not work as expected with scipenn
# use 'daydonor' instead
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_train], protein_trainsets = [adata_protein_train], 
                      gene_test = adata_gene_test, train_batchkeys = ['daydonor'], test_batchkey = 'daydonor',  use_gpu=False,
                     select_hvg=False)

Using CPU

QC Filtering Training Cells
QC Filtering Testing Cells

QC Filtering Training Genes
QC Filtering Testing Genes

Normalizing Training Cells
Normalizing Testing Cells


/home/asmauger/.conda/envs/scipenn_env/lib/python3.7/site-packages/scanpy/preprocessing/_normalization.py:182: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))



Log-Normalizing Training Data
Log-Normalizing Testing Data

Normalizing Gene Training Data by Batch


100%|██████████| 9/9 [00:00<00:00,  9.81it/s]



Normalizing Protein Training Data by Batch


100%|██████████| 9/9 [00:00<00:00, 16.02it/s]



Normalizing Gene Testing Data by Batch


100%|██████████| 9/9 [00:00<00:00, 10.56it/s]


In [21]:
sciPENN.train(quantiles = [0.1, 0.25, 0.75, 0.9], n_epochs = 10000, ES_max = 12, decay_max = 6, 
             decay_step = 0.1, lr = 10**(-3), load = True)

Epoch 0 prediction loss = 1.395
Epoch 1 prediction loss = 0.993
Epoch 2 prediction loss = 0.991
Epoch 3 prediction loss = 0.978
Epoch 4 prediction loss = 0.976
Epoch 5 prediction loss = 0.976
Epoch 6 prediction loss = 0.973
Epoch 7 prediction loss = 0.976
Epoch 8 prediction loss = 0.976
Decaying loss to 0.0001
Epoch 9 prediction loss = 0.967
Epoch 10 prediction loss = 0.965
Epoch 11 prediction loss = 0.966
Epoch 12 prediction loss = 0.966
Epoch 13 prediction loss = 0.965
Epoch 14 prediction loss = 0.964
Decaying loss to 1e-05
Epoch 15 prediction loss = 0.966
Epoch 16 prediction loss = 0.966
Epoch 17 prediction loss = 0.966
Epoch 18 prediction loss = 0.964
Epoch 19 prediction loss = 0.964
Epoch 20 prediction loss = 0.969
Decaying loss to 1.0000000000000002e-06
Epoch 21 prediction loss = 0.967


In [1]:
sciPENN

NameError: name 'sciPENN' is not defined

In [22]:
predicted_test = sciPENN.predict()

In [26]:
# predicted protein expression
predicted_test.X

array([[ 0.15121748,  0.33904052,  0.5782291 , ...,  0.27637747,
        -0.49629128,  0.15581037],
       [ 0.05389394,  0.19597393,  0.3290421 , ...,  0.13983977,
        -0.11790419,  0.10345943],
       [ 0.53248   ,  0.23699027,  0.2946961 , ...,  0.5570383 ,
        -0.43868726,  0.64300776],
       ...,
       [-0.38364536, -0.781911  , -1.4052956 , ..., -0.89036596,
         1.4259529 , -1.0780995 ],
       [-0.40152654, -0.71844053, -0.9645923 , ..., -0.7365623 ,
         1.1618565 , -0.53812027],
       [-0.20565024, -0.40426388, -0.41473943, ..., -0.12116696,
         0.10742351,  0.12652639]], dtype=float32)